In [ ]:
import qpas
import os
import scipy
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
import h5py
import time
import glob
from scipy.signal import hilbert, tukey
from scipy.fftpack import fft, rfft, irfft, rfftfreq
from scipy.optimize import least_squares
from numba import njit, prange
from skimage.transform import resize

### WORKFLOW ###
# (0) SET DRIVE PATH
# (1) SET META DATA

# then run the cells below to:
# (2) correct data for 
#     - pinout of the transducer (if applicable)
#     - timing jitter in the DAQ trigger 
# (3) (re-)order frames 
#     - ill: leftmost to rightmost
#     - wl: lowest to highest 
# (4) bandpass data
# (5) DAS beamform data
# (6) create resampled BMode image of the PA data
# (7) create result videos

### (0) SET DRIVE PATH ###
# to the raw PA recording data to be processed 
DRIVE_PATH = "/SET_PATH_TO_DATA/"

### (1) SET META DATA ###
# raw params
NO_ILL_POS = 4
NO_WAVELENGTHS = 10
NO_SEQUENCES_IN_FILE = 10
NO_FILES = 30
USE_PHOTOSOUND_AMP = False
USE_US = False

# reconstruction params
SAMPLE_RATE = 4.8077*4e6
PIXEL_CNT_X = 512
PIXEL_CNT_Y = 512-128
SENSOR_WIDTH = 38.4
MIN_DEPTH = SENSOR_WIDTH/8
MAX_DEPTH = SENSOR_WIDTH*7/8
SPEED_OF_SOUND = 1480.0
BMODE_RESOLUTION = 0.15

# relevant for pulse energy correction: wavelengths in nm
#WL = np.arange(680,981,20)
WL = np.asarray([760, 780, 800, 820, 840,
                 860, 880, 900, 920, 980])

acquisitions = glob.glob(DRIVE_PATH+"acquisition_*")

print("will process: "+str(acquisitions))


In [ ]:
# (2) correct data for 
#     - pinout of the transducer (if applicable)
#     - timing jitter in the DAQ trigger 
# (3) (re-)order frames 
#     - ill: leftmost to rightmost
#     - wl: lowest to highest

for acquisition in acquisitions:
    for i_ in range(NO_FILES):
        foo = scipy.io.loadmat(acquisition+'/rawdata_'+str(i_)+'.mat')
        raw_buffer = foo["rawbuffer_i"]
        raw_buffer = np.swapaxes(raw_buffer,0,3)
        raw_buffer = np.swapaxes(raw_buffer,1,2)[:,:,:,:]
        print("[loaded]", acquisition, str(i_), "with shape", raw_buffer.shape)
        if USE_PHOTOSOUND_AMP:
            pinout = np.asarray([126, 64, 62, 127, 124, 60, 57, 120, 122, 52, 56, 125, 51, 45, 53, 50, 49, 47, 46, 43, 38, 
                                 35, 34, 48, 41, 40, 39, 42, 44, 33, 37, 36, 29, 28, 21, 26, 23, 32, 25, 24, 17, 31, 30, 
                                 27, 22, 19, 18, 16, 15, 8, 20, 14, 68, 9, 65, 72, 73, 13, 12, 69, 4, 3, 5, 67, 113, 63, 
                                 61, 128, 59, 58, 119, 116, 115, 117, 55, 54, 121, 123, 118, 114, 108, 107, 110, 109, 102, 
                                 101, 99, 106, 103, 111, 97, 100, 98, 112, 105, 104, 89, 82, 81, 95, 93, 96, 88, 90, 87, 
                                 94, 92, 91, 84, 83, 86, 85, 79, 75, 70, 71, 11, 10, 76, 78, 77, 74, 7, 6, 66, 2, 1, 80])-1
            raw_buffer = raw_buffer[:,:,pinout,:]
        if USE_US:
            raw_buffer_us = raw_buffer[:,1::2,:,:]
            raw_buffer = raw_buffer[:,::2,:,:]
            raw_us_sorted = np.zeros([raw_buffer[0,0,:,0].size,
                                      raw_buffer[0,0,0,:].size,
                                      NO_SEQUENCES_IN_FILE,
                                      NO_WAVELENGTHS,
                                      NO_ILL_POS])
            for i_seq in range(NO_SEQUENCES_IN_FILE):
                for i_wav in range(NO_WAVELENGTHS):
                    for i_ill in range(NO_ILL_POS):
                        raw_us_sorted[:,:,i_seq,i_wav,i_ill] = raw_buffer_us[i_seq, i_ill+i_wav*NO_ILL_POS,:,:]

        raw_pa_sorted = np.zeros([raw_buffer[0,0,:,0].size,
                                  raw_buffer[0,0,0,:].size-110,
                                  NO_SEQUENCES_IN_FILE,
                                  NO_WAVELENGTHS,
                                  NO_ILL_POS])

        fig, ax = plt.subplots(1,3, figsize=(9,3))
        # correcting for laser pulse timing jitter
        for i_seq in range(NO_SEQUENCES_IN_FILE):
            for i_wav in range(NO_WAVELENGTHS):
                for i_ill in range(NO_ILL_POS):
                    envelope = np.abs(hilbert(np.mean(
                        raw_buffer[i_seq, i_ill + i_wav*NO_ILL_POS,:,30:110], 
                        axis=0),axis=0))
                    transducer_max = np.argmax(envelope[:])
                    ax[0].plot(np.abs(hilbert(np.mean(
                        raw_buffer[i_seq, i_ill + i_wav*NO_ILL_POS,:,0:400], 
                        axis=0),axis=0)))
                    jitter_corrected = raw_buffer[i_seq, i_ill + i_wav*NO_ILL_POS, :, 
                                                  transducer_max:raw_buffer[0,0,0,:].size+transducer_max-110]
                    raw_pa_sorted[:, :, i_seq, i_wav, i_ill] = jitter_corrected

        print("last transducer_max:" + str(transducer_max))
        del raw_buffer

        std_transducer_left_m_right = (np.std(raw_pa_sorted[0:10, :200, ...], axis=(1,0)) 
                                       - np.std(raw_pa_sorted[118:128, :200, ...], axis=(1,0)))
        mean_transducer = np.mean(np.abs(raw_pa_sorted[:, 20:50, ...]), axis=(1,0))
        mean_transducer_delta = np.zeros_like(mean_transducer)

        for k in range(NO_ILL_POS):
            for i in range(NO_SEQUENCES_IN_FILE):
                for j in range(NO_WAVELENGTHS):
                    mean_transducer_delta[i,j,k] = (mean_transducer[i,j,k] 
                                                    - mean_transducer[i,(j-1)%NO_WAVELENGTHS,k])        

            roll_wavelength_seq = np.argmax(np.mean(mean_transducer_delta[:,:,k], axis=(0)))
            print("start index for lowest wavelength", roll_wavelength_seq)

            # reorder wavelengths by rolling
            mean_transducer[:,:,k] = np.roll(mean_transducer[:,:,k], -roll_wavelength_seq, axis = 1)
            mean_transducer_delta[:,:,k] = np.roll(mean_transducer_delta[:,:,k], -roll_wavelength_seq, axis = 1)
            std_transducer_left_m_right[:,:,k] = np.roll(std_transducer_left_m_right[:,:,k], -roll_wavelength_seq, axis = 1)
            raw_pa_sorted[:,:,:,:,k] = np.roll(raw_pa_sorted[:,:,:,:,k], -roll_wavelength_seq, axis = 3)
#            if USE_US:
#                raw_us_sorted[:,:,:,:,k] = np.roll(raw_us_sorted[:,:,:,:,k], -roll_wavelength_seq, axis = 3)

            roll_wavelength_seq = np.argmax(np.mean(mean_transducer_delta[:,:,k], axis=(0)))
            print("start index for lowest wavelength after rolling", roll_wavelength_seq)

        roll_illumination_seq = np.argmax(np.mean(std_transducer_left_m_right, axis=(0,1)))
        print("start index for leftmost illumination", roll_illumination_seq)

        # reorder illumination by rolling
        mean_transducer = np.roll(mean_transducer, -roll_illumination_seq, axis = 2)
        mean_transducer_delta = np.roll(mean_transducer_delta, -roll_illumination_seq, axis = 2)
        std_transducer_left_m_right = np.roll(std_transducer_left_m_right, -roll_illumination_seq, axis = 2)
        raw_pa_sorted = np.roll(raw_pa_sorted, -roll_illumination_seq, axis = 4)
#        if USE_US:
#            raw_us_sorted = np.roll(raw_us_sorted, -roll_illumination_seq, axis = 4)

        roll_illumination_seq = np.argmax(np.mean(std_transducer_left_m_right, axis=(0,1)))
        print("start index for leftmost illumination after rolling", roll_illumination_seq)
        # visual check for issues / framedrops

        std_transducer_left_m_right = (np.std(raw_pa_sorted[0:10, :200, ...], axis=(1,0)) 
                                       - np.std(raw_pa_sorted[118:128, :200, ...], axis=(1,0)))

        colors = ["r","g","b","y"]
        for k in range(NO_ILL_POS):
            for i in range(NO_SEQUENCES_IN_FILE):
                ax[1].plot(std_transducer_left_m_right[i,:,k], color=colors[k], alpha=0.8)

        mean_transducer = np.mean(np.abs(raw_pa_sorted[:, 20:50, ...]), axis=(1,0))
        for k in range(NO_ILL_POS):
                for i in range(NO_SEQUENCES_IN_FILE):
                    ax[2].plot(mean_transducer[i,:,k], color=colors[k], alpha=0.8)

        with h5py.File(acquisition+'/rawdata_'+str(i_)+'_pa.h5', 'w') as f:
            f.create_dataset('rawbuffer', data=raw_pa_sorted)
            f.close()
        if USE_US:
            with h5py.File(acquisition+'/rawdata_'+str(i_)+'_us.h5', 'w') as f:
                f.create_dataset('rawbuffer', data=raw_us_sorted)
                f.close()

        plt.show()
        print("[done]")

In [ ]:
# (4) bandpass data
# (5) DAS beamform data
# (6) create resampled BMode image of the PA data

bmode_result_pa = np.zeros([256,256-64,NO_SEQUENCES_IN_FILE,NO_WAVELENGTHS,NO_ILL_POS])
for path in acquisitions:
    for i_ in range(NO_FILES):
        print(path)
        with h5py.File(path+"/rawdata_"+str(i_)+"_pa.h5", 'r') as f:
            raw_buffer_pa = np.asarray(f['rawbuffer'])
            f.close
        print("[loaded]", "with shape", raw_buffer_pa.shape)

        bf = qpas.beamformer(sample_rate=SAMPLE_RATE, 
                             speed_of_sound=SPEED_OF_SOUND, 
                             modality="PA")
        bf.set_linear_sensor()
        bf.set_output_area(x0=0, x1=SENSOR_WIDTH, 
                           min_depth=MIN_DEPTH, max_depth=MAX_DEPTH,
                           pixel_count_x=PIXEL_CNT_X, 
                           pixel_count_y=PIXEL_CNT_Y)
        bf.set_apodization(func = "Hann", angle = 30)

        for i_seq in range(NO_SEQUENCES_IN_FILE):
            raw_signal_PA = qpas.tukeybandpass(raw_buffer_pa[:,:,i_seq,:,:],
                                               sample_rate=SAMPLE_RATE)
            bf_image_PA = bf.beamform(signal=raw_signal_PA)
            bmode_image_PA = qpas.bmode(bf_image_PA)
            bmode_result_pa[:,:,i_seq,:,:] = resize(bmode_image_PA, 
                                                    [bmode_result_pa.shape[0],
                                                     bmode_result_pa.shape[1],
                                                     NO_WAVELENGTHS,NO_ILL_POS])
            print(i_seq, "/", NO_SEQUENCES_IN_FILE)

        with h5py.File(path+'/bmode_image_'+str(i_)+'_PA_.h5', 'w') as f:
            f.create_dataset('bmode_image_PA', data=bmode_result_pa)
            f.close()

In [ ]:
if USE_US:
    bmode_result_us = np.zeros([256,256-64,NO_SEQUENCES_IN_FILE,NO_WAVELENGTHS,NO_ILL_POS])
    for path in acquisitions:
        for i_ in range(NO_FILES):
            print(path)
            with h5py.File(path+"/rawdata_"+str(i_)+"_us.h5", 'r') as f:
                raw_buffer_us = np.asarray(f['rawbuffer'])
                f.close
            print("[loaded]", "with shape", raw_buffer_us.shape)

            bf = qpas.beamformer(sample_rate=SAMPLE_RATE, 
                                 speed_of_sound=SPEED_OF_SOUND, 
                                 modality="plane wave US")
            bf.set_linear_sensor()
            bf.set_output_area(x0=0, x1=SENSOR_WIDTH, 
                               min_depth=MIN_DEPTH, max_depth=MAX_DEPTH,
                               pixel_count_x=PIXEL_CNT_X, 
                               pixel_count_y=PIXEL_CNT_Y)
            bf.set_apodization(func = "Hann", angle = 30)

            for i_seq in range(NO_SEQUENCES_IN_FILE):
                raw_signal_US = np.nan_to_num(raw_buffer_us[:,:,i_seq,:,:])
                bf_image_US = bf.beamform(signal=raw_signal_US)
                bmode_image_US = qpas.bmode(np.nan_to_num(bf_image_US))
                bmode_result_us[:,:,i_seq,:,:] = resize(bmode_image_US, 
                                                        [bmode_result_us.shape[0],
                                                         bmode_result_us.shape[1],
                                                         NO_WAVELENGTHS,NO_ILL_POS])
                print(i_seq, "/", NO_SEQUENCES_IN_FILE)

            with h5py.File(path+'/bmode_image_'+str(i_)+'_US.h5', 'w') as f:
                f.create_dataset('bmode_image_US', data=bmode_result_us)
                f.close()

In [ ]:
ROLLING_AVRG_OVER = 5

for path in acquisitions:
    print(path)
    with h5py.File(path+"/bmode_image_0_PA_.h5", 'r') as f:
        pa = np.asarray(f['bmode_image_PA'])
        f.close
    fig, ax = plt.subplots(1, 4, figsize=(8,2), dpi=300)
    for i in range(NO_ILL_POS):
        ax[i].imshow(20*np.log10(pa[:,:160,0,0,i]
                         /np.max(pa[:,:160,0,0,i])).T, 
                     aspect=1, vmin=-50, vmax=0)
    plt.show()
    
    titel = "rolling mean PA B-Mode [db]"
    pa_avg_ill = np.zeros([256,100,NO_WAVELENGTHS,NO_SEQUENCES_IN_FILE*NO_FILES])
    for i_f in range(NO_FILES):
        with h5py.File(path+"/bmode_image_"+str(i_f)+"_PA_.h5", 'r') as f:
            pa = np.asarray(f['bmode_image_PA'])
            f.close
        for i_i in range(NO_SEQUENCES_IN_FILE):
            pa_avg_ill[:,:,:,i_f*NO_SEQUENCES_IN_FILE+i_i] = np.mean(pa[:,68:168,i_i,:,:], axis=3)

    pa_rolling_avg_ill = np.zeros([256,100,NO_WAVELENGTHS,NO_SEQUENCES_IN_FILE*NO_FILES-ROLLING_AVRG_OVER+1])
    for i_ in range(NO_SEQUENCES_IN_FILE*NO_FILES-ROLLING_AVRG_OVER+1):
        pa_rolling_avg_ill[:,:,:,i_] = np.mean(pa_avg_ill[:,:,:,i_:i_+ROLLING_AVRG_OVER], axis=3)

    qpas.create_vid(inputData=np.mean(pa_rolling_avg_ill, axis=(2)), filename=path+"/rolling_mean", 
                    titel=titel, resolution=BMODE_RESOLUTION, vmin=-50, vmax=0, fps = 24, cmap = cm.viridis, 
                    normalize_frame = 'Log', aspect = 1)
    
    titel = "full sequence PA B-Mode real speed [db]"
    pa_raw_seq = np.zeros([256,100,NO_SEQUENCES_IN_FILE*NO_FILES*NO_WAVELENGTHS*NO_ILL_POS])
    for i_f in range(NO_FILES):
        with h5py.File(path+"/bmode_image_"+str(i_f)+"_PA_.h5", 'r') as f:
            pa = np.asarray(f['bmode_image_PA'])
            f.close
        for i_i in range(NO_SEQUENCES_IN_FILE):
            for i_wl in range(NO_WAVELENGTHS):
                _i = (i_f*NO_SEQUENCES_IN_FILE*NO_WAVELENGTHS*NO_ILL_POS
                      + i_i*NO_WAVELENGTHS*NO_ILL_POS + i_wl*NO_ILL_POS)
                pa_raw_seq[:,:,_i:_i+NO_ILL_POS] = pa[:,68:168,i_i,i_wl,:]

    ## WARNING: rendering this video takes forever!
    #qpas.create_vid(inputData= pa_raw_seq, filename=path+"/full_sequence", 
    #                titel=titel, resolution=BMODE_RESOLUTION, vmin=-60, vmax=0, fps = 100, cmap = cm.viridis, 
    #                normalize_frame = 'Log', aspect = 1)

In [ ]:
if USE_US:
    for path in acquisitions:
        print(path)
        titel = "full sequence US B-Mode real speed [db]"
        us_raw_seq = np.zeros([256,100,NO_SEQUENCES_IN_FILE*NO_FILES*NO_WAVELENGTHS*NO_ILL_POS])
        for i_f in range(NO_FILES):
            with h5py.File(path+"/bmode_image_"+str(i_f)+"_US.h5", 'r') as f:
                us = np.asarray(f['bmode_image_US'])
                f.close
            for i_i in range(NO_SEQUENCES_IN_FILE):
                for i_wl in range(NO_WAVELENGTHS):
                    _i = (i_f*NO_SEQUENCES_IN_FILE*NO_WAVELENGTHS*NO_ILL_POS
                          + i_i*NO_WAVELENGTHS*NO_ILL_POS + i_wl*NO_ILL_POS)
                    us_raw_seq[:,:,_i:_i+NO_ILL_POS] = np.nan_to_num(us[:,62:162,i_i,i_wl,:])

        qpas.create_vid(inputData=us_raw_seq, filename=path+"/full_sequence_us", 
                        titel=titel, resolution=BMODE_RESOLUTION, vmin=-60, vmax=0, fps = 100, cmap = cm.gray, 
                        normalize_frame = 'Log', aspect = 1)